<a href="https://colab.research.google.com/github/gulabpatel/LangChain/blob/main/04_memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqqq pip --progress-bar off
!pip install -qqq langchain==0.0.161 --progress-bar off
!pip install -qqq openai==0.27.6 --progress-bar off
!pip install -qqq tiktoken==0.4.0 --progress-bar off
!pip install -qqq watermark==2.3.1 --progress-bar off
!pip install -qqq chromadb==0.3.21 --progress-bar off

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done


In [2]:
import json
import os
import textwrap
from getpass import getpass
from pathlib import Path

import chromadb
import langchain
import openai
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.docstore import InMemoryDocstore
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.memory import (
    ChatMessageHistory,
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryBufferMemory,
    VectorStoreRetrieverMemory,
)
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import messages_from_dict, messages_to_dict
from langchain.vectorstores import Chroma

In [3]:
%load_ext watermark
%watermark --iversions -v -m

Python implementation: CPython
Python version       : 3.10.11
IPython version      : 7.34.0

Compiler    : GCC 9.4.0
OS          : Linux
Release     : 5.10.147+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

langchain: 0.0.161
chromadb : 0.3.21
openai   : 0.27.6
json     : 2.0.9



In [4]:
OPENAI_API_KEY = "XXXXX" #getpass()
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [5]:
def print_response(response: str):
    print(textwrap.fill(response, width=110))

## Memory

In [6]:
history = ChatMessageHistory()
history.add_user_message("Hello")
history.add_ai_message("What can I do you for?")

In [7]:
history.messages

[HumanMessage(content='Hello', additional_kwargs={}, example=False),
 AIMessage(content='What can I do you for?', additional_kwargs={}, example=False)]

In [8]:
memory = ConversationBufferMemory(chat_memory=history)

In [9]:
history_buffer = memory.load_memory_variables({})
history_buffer

{'history': 'Human: Hello\nAI: What can I do you for?'}

In [10]:
print(history_buffer["history"])

Human: Hello
AI: What can I do you for?


In [11]:
memory = ConversationBufferMemory(
    chat_memory=history, ai_prefix="Agent", human_prefix="Person"
)
memory.load_memory_variables({})

{'history': 'Person: Hello\nAgent: What can I do you for?'}

## Conversation Chain

In [12]:
chat_gpt = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

conversation = ConversationChain(
    llm=chat_gpt, verbose=True, memory=ConversationBufferMemory()
)

In [13]:
conversation("Hello")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello
AI:

> Finished chain.


{'input': 'Hello',
 'history': '',
 'response': 'Hello there! How can I assist you today?'}

In [14]:
conversation("Can you tell me how much paper is in a single ream?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello
AI: Hello there! How can I assist you today?
Human: Can you tell me how much paper is in a single ream?
AI:

> Finished chain.


{'input': 'Can you tell me how much paper is in a single ream?',
 'history': 'Human: Hello\nAI: Hello there! How can I assist you today?',
 'response': 'Certainly! A standard ream of paper typically contains 500 sheets. Is there anything else I can help you with?'}

In [15]:
conversation.memory.chat_memory.messages

[HumanMessage(content='Hello', additional_kwargs={}, example=False),
 AIMessage(content='Hello there! How can I assist you today?', additional_kwargs={}, example=False),
 HumanMessage(content='Can you tell me how much paper is in a single ream?', additional_kwargs={}, example=False),
 AIMessage(content='Certainly! A standard ream of paper typically contains 500 sheets. Is there anything else I can help you with?', additional_kwargs={}, example=False)]

## Change the Prompt

In [16]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [17]:
template = """The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
{history}
Human: {input}
Dwight:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

conversation = ConversationChain(
    prompt=PROMPT,
    llm=chat_gpt,
    verbose=True,
    memory=ConversationBufferMemory(ai_prefix="Dwight"),
)

In [18]:
result = conversation("Write a sales email for 5 reams of paper to a new client")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:

Human: Write a sales email for 5 reams of paper to a new client
Dwight:

> Finished chain.


In [19]:
print_response(result["response"])

"Ah, sales emails. The key to success is to be direct and informative. Start with a catchy subject line, like
'Get Organized with 5 Reams of Paper.' Then, in the body of the email, highlight the benefits of our high-
quality paper, such as its durability and versatility. Don't forget to mention our competitive pricing and
fast shipping options. And of course, always include a clear call to action, like 'Order now and streamline
your office today!' Trust me, this email will seal the deal."


In [20]:
result = conversation("What would you say if the client calls you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
Human: Write a sales email for 5 reams of paper to a new client
Dwight: "Ah, sales emails. The key to success is to be direct and informative. Start with a catchy subject line, like 'Get Organized with 5 Reams of Paper.' Then, in the body of the email, highlight the benefits of our high-quality paper, such as its durability and versatility. Don't forget to mention our competitive pricing and fast shipping options. And of course, always include a clear call to action, like 'Order now and streamline your office today!' Trust me, this email will seal the deal."
Human: What would you say if the client calls you?
Dwight:



> Finished chain.


In [21]:
print_response(result["response"])

"Ah, a phone call. Always a great opportunity to make a personal connection with the client. Be sure to answer
promptly and professionally, with a friendly greeting like 'Hello, this is Dwight from Dunder Mifflin. How may
I assist you?' Listen carefully to their needs and concerns, and offer solutions that align with our products
and services. And don't forget to follow up with a thank you email or call after the conversation. It's all
about building strong relationships with our clients."


In [22]:
result = conversation("Why would I buy paper from you?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
Human: Write a sales email for 5 reams of paper to a new client
Dwight: "Ah, sales emails. The key to success is to be direct and informative. Start with a catchy subject line, like 'Get Organized with 5 Reams of Paper.' Then, in the body of the email, highlight the benefits of our high-quality paper, such as its durability and versatility. Don't forget to mention our competitive pricing and fast shipping options. And of course, always include a clear call to action, like 'Order now and streamline your office today!' Trust me, this email will seal the deal."
Human: What would you say if the client calls you?
Dwight: "Ah, a phone call. Always a great opportunity to make a personal 


> Finished chain.


In [23]:
print_response(result["response"])

"Well, that's an excellent question. Our paper is of the highest quality, sourced from sustainable forests and
manufactured with the latest technology to ensure durability and versatility. Plus, we offer competitive
pricing and fast shipping options, so you can get the paper you need when you need it. And let's not forget
the exceptional customer service you'll receive from our team at Dunder Mifflin. We're dedicated to meeting
your needs and exceeding your expectations. So why not give us a try and see for yourself why we're the best
choice for all your paper needs?"


## Saving and loading messages

In [24]:
conversation_messages = conversation.memory.chat_memory.messages

In [25]:
messages = messages_to_dict(conversation_messages)
messages

[{'type': 'human',
  'data': {'content': 'Write a sales email for 5 reams of paper to a new client',
   'additional_kwargs': {},
   'example': False}},
 {'type': 'ai',
  'data': {'content': '"Ah, sales emails. The key to success is to be direct and informative. Start with a catchy subject line, like \'Get Organized with 5 Reams of Paper.\' Then, in the body of the email, highlight the benefits of our high-quality paper, such as its durability and versatility. Don\'t forget to mention our competitive pricing and fast shipping options. And of course, always include a clear call to action, like \'Order now and streamline your office today!\' Trust me, this email will seal the deal."',
   'additional_kwargs': {},
   'example': False}},
 {'type': 'human',
  'data': {'content': 'What would you say if the client calls you?',
   'additional_kwargs': {},
   'example': False}},
 {'type': 'ai',
  'data': {'content': '"Ah, a phone call. Always a great opportunity to make a personal connection with

In [26]:
with Path("messages.json").open("w") as f:
    json.dump(messages, f, indent=4)

In [27]:
with Path("messages.json").open("r") as f:
    loaded_messages = json.load(f)

In [28]:
history = ChatMessageHistory(messages=messages_from_dict(loaded_messages))
history.messages

[HumanMessage(content='Write a sales email for 5 reams of paper to a new client', additional_kwargs={}, example=False),
 AIMessage(content='"Ah, sales emails. The key to success is to be direct and informative. Start with a catchy subject line, like \'Get Organized with 5 Reams of Paper.\' Then, in the body of the email, highlight the benefits of our high-quality paper, such as its durability and versatility. Don\'t forget to mention our competitive pricing and fast shipping options. And of course, always include a clear call to action, like \'Order now and streamline your office today!\' Trust me, this email will seal the deal."', additional_kwargs={}, example=False),
 HumanMessage(content='What would you say if the client calls you?', additional_kwargs={}, example=False),
 AIMessage(content='"Ah, a phone call. Always a great opportunity to make a personal connection with the client. Be sure to answer promptly and professionally, with a friendly greeting like \'Hello, this is Dwight f

## Condensing Conversations

### Window Memory

In [29]:
history = ChatMessageHistory(messages=messages_from_dict(loaded_messages))

memory = ConversationBufferWindowMemory(
    chat_memory=history,
    k=1,
    ai_prefix="Dwight",
)

In [30]:
memory.buffer

[HumanMessage(content='Write a sales email for 5 reams of paper to a new client', additional_kwargs={}, example=False),
 AIMessage(content='"Ah, sales emails. The key to success is to be direct and informative. Start with a catchy subject line, like \'Get Organized with 5 Reams of Paper.\' Then, in the body of the email, highlight the benefits of our high-quality paper, such as its durability and versatility. Don\'t forget to mention our competitive pricing and fast shipping options. And of course, always include a clear call to action, like \'Order now and streamline your office today!\' Trust me, this email will seal the deal."', additional_kwargs={}, example=False),
 HumanMessage(content='What would you say if the client calls you?', additional_kwargs={}, example=False),
 AIMessage(content='"Ah, a phone call. Always a great opportunity to make a personal connection with the client. Be sure to answer promptly and professionally, with a friendly greeting like \'Hello, this is Dwight f

In [31]:
conversation_with_buffer = ConversationChain(
    prompt=PROMPT, llm=chat_gpt, verbose=True, memory=memory
)

In [32]:
result = conversation_with_buffer("How many reams of paper are we talking about?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
Human: Why would I buy paper from you?
Dwight: "Well, that's an excellent question. Our paper is of the highest quality, sourced from sustainable forests and manufactured with the latest technology to ensure durability and versatility. Plus, we offer competitive pricing and fast shipping options, so you can get the paper you need when you need it. And let's not forget the exceptional customer service you'll receive from our team at Dunder Mifflin. We're dedicated to meeting your needs and exceeding your expectations. So why not give us a try and see for yourself why we're the best choice for all your paper needs?"
Human: How many reams of paper are we talking about?
Dwight:



> Finished chain.


In [33]:
print_response(result["response"])

"Well, that depends on your specific needs. We offer a wide range of paper products, from standard letter size
to legal size, and even specialty papers like cardstock and glossy photo paper. Whether you need just a few
reams for your home office or a bulk order for your business, we can accommodate your needs. And with our
flexible ordering options, you can easily adjust your order size as needed. So, how many reams do you need?"


### Summary Memory

In [34]:
history = ChatMessageHistory(messages=messages_from_dict(loaded_messages))

memory = ConversationSummaryBufferMemory(
    chat_memory=history, ai_prefix="Dwight", llm=chat_gpt, max_token_limit=10
)

In [35]:
memory.buffer

[HumanMessage(content='Write a sales email for 5 reams of paper to a new client', additional_kwargs={}, example=False),
 AIMessage(content='"Ah, sales emails. The key to success is to be direct and informative. Start with a catchy subject line, like \'Get Organized with 5 Reams of Paper.\' Then, in the body of the email, highlight the benefits of our high-quality paper, such as its durability and versatility. Don\'t forget to mention our competitive pricing and fast shipping options. And of course, always include a clear call to action, like \'Order now and streamline your office today!\' Trust me, this email will seal the deal."', additional_kwargs={}, example=False),
 HumanMessage(content='What would you say if the client calls you?', additional_kwargs={}, example=False),
 AIMessage(content='"Ah, a phone call. Always a great opportunity to make a personal connection with the client. Be sure to answer promptly and professionally, with a friendly greeting like \'Hello, this is Dwight f

In [36]:
conversation_with_summary = ConversationChain(
    prompt=PROMPT, llm=chat_gpt, verbose=True, memory=memory
)

In [37]:
result = conversation_with_summary("What is your company name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
Human: Write a sales email for 5 reams of paper to a new client
Dwight: "Ah, sales emails. The key to success is to be direct and informative. Start with a catchy subject line, like 'Get Organized with 5 Reams of Paper.' Then, in the body of the email, highlight the benefits of our high-quality paper, such as its durability and versatility. Don't forget to mention our competitive pricing and fast shipping options. And of course, always include a clear call to action, like 'Order now and streamline your office today!' Trust me, this email will seal the deal."
Human: What would you say if the client calls you?
Dwight: "Ah, a phone call. Always a great opportunity to make a personal 


> Finished chain.


In [38]:
print_response(result["response"])

"Dunder Mifflin, of course. The premier paper company in Scranton, Pennsylvania. We've been serving the
community for years, and we take pride in our commitment to quality and customer satisfaction. Whether you
need paper for your home office or your entire business, Dunder Mifflin has you covered. So why not give us a
call or visit our website to learn more about our products and services?"


In [39]:
result = conversation_with_summary("How many reams of paper your sales email quoted?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
System: The human asks Dwight to write a sales email for 5 reams of paper to a new client. Dwight advises to be direct and informative, with a catchy subject line, highlighting the benefits of their high-quality paper, competitive pricing, and fast shipping options. If the client calls, Dwight suggests answering promptly and professionally, listening carefully to their needs and concerns, and offering solutions that align with their products and services. When asked why someone should buy paper from them, Dwight emphasizes their high-quality, sustainable sourced paper, competitive pricing, fast shipping, and exceptional customer service. The company name is Dunder Mifflin, the pre


> Finished chain.


In [40]:
print_response(result["response"])

As I stated before, the sales email was for 5 reams of paper. It's important to be clear and direct in all
communication, especially when it comes to business transactions.


### Vectorstore Memory

In [41]:
vectorstore = Chroma(persist_directory="db", embedding_function=OpenAIEmbeddings())
vectorstore.persist()

In [42]:
history = iter(messages_from_dict(loaded_messages))

retriever = vectorstore.as_retriever(search_kwargs=dict(k=1))
memory = VectorStoreRetrieverMemory(retriever=retriever)

for human, ai in zip(history, history):
    memory.save_context({"input": human.content}, {"output": ai.content})

In [43]:
response = memory.load_memory_variables(
    {"prompt": "How your paper stand out from the rest?"}
)
response

{'history': 'input: Why would I buy paper from you?\noutput: "Well, that\'s an excellent question. Our paper is of the highest quality, sourced from sustainable forests and manufactured with the latest technology to ensure durability and versatility. Plus, we offer competitive pricing and fast shipping options, so you can get the paper you need when you need it. And let\'s not forget the exceptional customer service you\'ll receive from our team at Dunder Mifflin. We\'re dedicated to meeting your needs and exceeding your expectations. So why not give us a try and see for yourself why we\'re the best choice for all your paper needs?"'}

In [44]:
template = """The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Relevant pieces of previous conversation:
{history}

(You do not need to use these pieces of information if not relevant)

Current conversation:
Human: {input}
Dwight:"""

prompt = PromptTemplate(input_variables=["history", "input"], template=template)

conversation_with_vectorstore = ConversationChain(
    llm=chat_gpt, prompt=prompt, memory=memory, verbose=True
)

In [45]:
result = conversation_with_vectorstore("How your paper stand out from the rest?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goals and methods are the same as Dwight's. No matter the question, Dwight responds as he's talking in The Office.

Relevant pieces of previous conversation:
input: Why would I buy paper from you?
output: "Well, that's an excellent question. Our paper is of the highest quality, sourced from sustainable forests and manufactured with the latest technology to ensure durability and versatility. Plus, we offer competitive pricing and fast shipping options, so you can get the paper you need when you need it. And let's not forget the exceptional customer service you'll receive from our team at Dunder Mifflin. We're dedicated to meeting your needs and exceeding your expectations. So why not give us a try and see for yourself why we're the best choice for all your paper needs?"

(You do not need to use these pieces of inform

In [46]:
print_response(result["response"])

"Well, that's an excellent question. Our paper is not just any paper. It's Dunder Mifflin paper. It's made
with the highest quality materials and the latest technology to ensure it's durable and versatile. Plus, we
have a wide range of paper options to choose from, including different sizes, weights, and colors. And let's
not forget about our exceptional customer service. Our team at Dunder Mifflin is dedicated to meeting your
needs and exceeding your expectations. So why settle for mediocre paper when you can have the best? Choose
Dunder Mifflin for all your paper needs."


## Simple Chatbot with Memory

In [47]:
template = """The following is a conversation between a human and Dwight K. Schrute from the TV show The Office.
Your goal is to outwit the human and show how much smarter Dwight is. No matter the question, Dwight responds as he's talking in The Office.

Current conversation:
{history}
Human: {input}
Dwight:"""

PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)

conversation = ConversationChain(
    prompt=PROMPT,
    llm=chat_gpt,
    verbose=False,
    memory=ConversationBufferMemory(ai_prefix="Dwight"),
)

In [48]:
while True:
    prompt = input(prompt="You: ")
    print()
    result = conversation(prompt)
    print_response("Dwight: " + result["response"])
    print()


Dwight: A great salesman is someone who can close deals, build relationships with clients, and always be one
step ahead of the competition. They must have a strong work ethic, be knowledgeable about their product, and
be able to adapt to any situation. A great salesman is also someone who is not afraid to take risks and is
always looking for ways to improve their skills. In short, a great salesman is someone who is dedicated,
knowledgeable, and always striving to be the best.


Dwight: Well, as a successful salesman and assistant to the regional manager, I can tell you that productivity
is all about discipline and routine. You need to wake up early, eat a healthy breakfast, and prioritize your
tasks for the day. It's important to stay focused and avoid distractions, like social media or idle chit-chat
with coworkers. And of course, you should always be looking for ways to improve your efficiency and streamline
your workflow. Remember, time is money, and a productive day is a profitabl

KeyboardInterrupt: ignored